In [9]:
import math
# from timber.trainer.timber_trainer import load_model
import torch
import os

_N_H = 3
TDST_BQ = 5
MASK_K_BK = 4
MODEL_N = 3
ensemble_final_timedim = 3

ensemble_attn_mask_per_layer = torch.randint(1, 11, (_N_H, TDST_BQ, MASK_K_BK, MODEL_N))

In [10]:
print(ensemble_attn_mask_per_layer.shape)
print(ensemble_attn_mask_per_layer)

TDST_BQ_TIMEDIM = TDST_BQ // ensemble_final_timedim
timedim_leftover = TDST_BQ % ensemble_final_timedim

if timedim_leftover > 0:
    TDST_BQ_TIMEDIM += 1
    padding_tensor = torch.full((_N_H, ensemble_final_timedim - timedim_leftover, MASK_K_BK, MODEL_N), 9999999)
    TDST_BQ += ensemble_final_timedim - timedim_leftover
    ensemble_attn_mask_per_layer = torch.cat((ensemble_attn_mask_per_layer, padding_tensor), dim=1)

assert ensemble_attn_mask_per_layer.shape == (_N_H, TDST_BQ, MASK_K_BK, MODEL_N)
ensemble_attn_mask_per_layer = ensemble_attn_mask_per_layer.view(_N_H, TDST_BQ, MASK_K_BK, MODEL_N)
ensemble_attn_mask_per_layer = ensemble_attn_mask_per_layer.view(_N_H * TDST_BQ, MASK_K_BK * MODEL_N)
assert TDST_BQ % ensemble_final_timedim == 0
ensemble_attn_mask_per_layer = ensemble_attn_mask_per_layer.view((_N_H * TDST_BQ)//ensemble_final_timedim, MASK_K_BK * MODEL_N * ensemble_final_timedim)
print(ensemble_attn_mask_per_layer.shape)
print(ensemble_attn_mask_per_layer)


# ensemble_attn_mask_per_layer : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
unique_x, indices, unique_cnt = torch.unique(ensemble_attn_mask_per_layer, return_inverse=True, sorted=False, return_counts=True)
# indices -= indices.min(dim=1, keepdims=True)[0]

# cnt_x = (unique_x[None, None, :] == ensemble_attn_mask_per_layer[:, :, None]).long().sum(1)
N, K = ensemble_attn_mask_per_layer.shape

cnt_xs = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    cnt_xs.append(
        (unique_x[None, None, :] == ensemble_attn_mask_per_layer[icn*chunk_n:icn*chunk_n+chunk_n, :, None]).long().sum(1)
    )
cnt_x = torch.cat(cnt_xs, dim=0)


result = torch.full_like(ensemble_attn_mask_per_layer, 9999999)
result = result.scatter_(1, indices.clamp(0, K-1), ensemble_attn_mask_per_layer)


# t = result[:, :, None] == unique_x[None, None, :]
# t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
# t = t.sum(-1)
N, K = result.shape
ts = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    t = result[icn*chunk_n:icn*chunk_n+chunk_n, :, None] == unique_x[None, None, :]
    t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
    t = t.sum(-1)
    ts.append(t)
t = torch.cat(ts, dim=0)

# torch.Size([4096, 1280]) torch.Size([64, 1280]) torch.Size([4096, 2094])
# print(result.shape, t.shape, cnt_x.shape)
result_cnt = torch.where(result < 9999999, cnt_x.gather(-1, t), -9999999)

'''
ensemble_attn_mask_per_layer
tensor([[1, 1, 3, 3, 3, 5],\
        [3, 3, 4, 4, 4, 4]])
ensemble_cnt_sorted
tensor([[       6,        4,        3,        3,        2,        1,        1,
-9999999, -9999999, -9999999, -9999999, -9999999, -9999999, -9999999,
-9999999, -9999999, -9999999, -9999999, -9999999, -9999999],\
[       6,        4,        3,        3,        2,        2, -9999999,
-9999999, -9999999, -9999999, -9999999, -9999999, -9999999, -9999999,
-9999999, -9999999, -9999999, -9999999, -9999999, -9999999]])
ensemble_sorted
tensor([[   10,     6,     3,     8,     1,     5,    11, 32000, 32000, 32000,
32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000],\
[    5,     4,     7,     9,     6,     3, 32000, 32000, 32000, 32000,
32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000]]))
'''
# ensemble_sorted : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
# ensemble_cnt_sorted : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
ensemble_cnt_sorted, indices = torch.sort(result_cnt, dim=-1, descending=True)
ensemble_sorted = result.gather(-1, indices)




torch.Size([3, 5, 4, 3])
tensor([[[[ 2,  1,  2],
          [10,  8,  1],
          [10,  5,  4],
          [ 1,  3,  5]],

         [[ 1,  9,  8],
          [ 4,  7,  9],
          [ 1,  6,  5],
          [ 1,  2,  9]],

         [[ 2,  4,  2],
          [ 6,  1,  1],
          [10,  7,  9],
          [ 7,  9,  6]],

         [[ 9,  8,  8],
          [ 7,  6,  5],
          [ 6,  3,  4],
          [ 1,  2,  9]],

         [[ 5, 10,  3],
          [10,  3,  5],
          [ 7,  6,  8],
          [ 8,  3,  1]]],


        [[[ 3,  1,  7],
          [ 5,  1,  1],
          [ 4,  7,  5],
          [ 4,  5,  9]],

         [[ 5, 10,  3],
          [ 2,  9,  5],
          [ 7,  7,  4],
          [ 3,  6,  5]],

         [[ 6,  7,  4],
          [10,  5,  6],
          [10, 10,  7],
          [ 7,  8,  3]],

         [[ 1,  3,  8],
          [10,  4,  6],
          [ 4,  7, 10],
          [ 7,  4,  1]],

         [[ 3,  2,  5],
          [ 3,  8,  2],
          [ 3,  7,  9],
          [ 4,  7, 

In [18]:
print(ensemble_sorted)
print(ensemble_cnt_sorted)
print(ensemble_sorted.shape)
print(ensemble_cnt_sorted.shape)

tensor([[      6,       2,       9,       3,       7,       8,       5,      10,
               1,       4, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999,
         9999999, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999,
         9999999, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999,
         9999999, 9999999, 9999999, 9999999],
        [      2,      10,       5,       1,       9,       3,       7,       8,
         9999999, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999,
         9999999, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999,
         9999999, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999,
         9999999, 9999999, 9999999, 9999999],
        [      3,       7,      10,       6,       8,       4,       1,       5,
               2,       9, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999,
         9999999, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999, 9999999,
         9999999,

In [17]:
t = torch.randn(2,5)

print(t)
print(t[:, :9])

tensor([[-0.0807, -0.4962,  0.7057, -2.9348, -1.8689],
        [ 0.2331,  0.7846, -0.8708, -1.1181, -2.6611]])
tensor([[-0.0807, -0.4962,  0.7057, -2.9348, -1.8689],
        [ 0.2331,  0.7846, -0.8708, -1.1181, -2.6611]])


In [ ]:
mask = ensemble_cnt_sorted >= ensemble_method_final_inter_thresh

ensemble_filtered = torch.where(mask, ensemble_sorted, torch.tensor(9999999, device=mask.device))
ensemble_cnt_filtered = torch.where(mask, ensemble_cnt_sorted, torch.tensor(9999999, device=mask.device))

## mask_i : where to discard leftovers 
filtered_mask = ensemble_filtered == 9999999
# Determine which columns have all rows as -1
columns_with_all_negative_one = torch.all(filtered_mask, dim=0)

# Get the first index where all rows have -1
nonzero_indices = torch.nonzero(columns_with_all_negative_one, as_tuple=True)

# If there are any such columns, find the first one
if len(nonzero_indices[0]) > 0:
    mask_k_i = nonzero_indices[0][0].item()
    k_final = min(mask_k_i, ensemble_indices_k_size)
else:
    mask_k_i = -1  # If no such index is found
    k_final = ensemble_indices_k_size

ensemble_filtered = ensemble_filtered[:, :k_final] # TODO is this meaningful??
ensemble_cnt_filtered = ensemble_cnt_filtered[:, :k_final]
ensemble_filtered = ensemble_filtered.view(_N_H, TDST_BQ, -1)

In [4]:
import torch
e = torch.randint(1, 10, (6, 4))
print(e)
e = e.view(6//3, 3*4)

print(e)

tensor([[2, 7, 8, 7],
        [1, 3, 5, 6],
        [9, 2, 9, 5],
        [1, 6, 7, 7],
        [3, 1, 5, 4],
        [7, 5, 8, 6]])
tensor([[2, 7, 8, 7, 1, 3, 5, 6, 9, 2, 9, 5],
        [1, 6, 7, 7, 3, 1, 5, 4, 7, 5, 8, 6]])


In [8]:
ensemble_attn_mask_per_layer = ensemble_attn_mask_per_layer.view(_N_H * TDST_BQ, MASK_K_BK * MODEL_N)
# ensemble_attn_mask_per_layer : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
unique_x, indices, unique_cnt = torch.unique(ensemble_attn_mask_per_layer, return_inverse=True, sorted=False, return_counts=True)
# indices -= indices.min(dim=1, keepdims=True)[0]

# cnt_x = (unique_x[None, None, :] == ensemble_attn_mask_per_layer[:, :, None]).long().sum(1)
N, K = ensemble_attn_mask_per_layer.shape

cnt_xs = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    cnt_xs.append(
        (unique_x[None, None, :] == ensemble_attn_mask_per_layer[icn*chunk_n:icn*chunk_n+chunk_n, :, None]).long().sum(1)
    )
cnt_x = torch.cat(cnt_xs, dim=0)


result = torch.full_like(ensemble_attn_mask_per_layer, 9999999)
result = result.scatter_(1, indices.clamp(0, K-1), ensemble_attn_mask_per_layer)


# t = result[:, :, None] == unique_x[None, None, :]
# t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
# t = t.sum(-1)
N, K = result.shape
ts = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    t = result[icn*chunk_n:icn*chunk_n+chunk_n, :, None] == unique_x[None, None, :]
    t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
    t = t.sum(-1)
    ts.append(t)
t = torch.cat(ts, dim=0)

# torch.Size([4096, 1280]) torch.Size([64, 1280]) torch.Size([4096, 2094])
# print(result.shape, t.shape, cnt_x.shape)
result_cnt = torch.where(result < 9999999, cnt_x.gather(-1, t), -9999999)

In [6]:
import torch

# Define the existing tensor with dimensions (_N_H, TDST_BQ, MASK_K_BK, MODEL_N)
_N_H = 3
TDST_BQ = 4
MASK_K_BK = 2
MODEL_N = 5

# Existing tensor
ensemble_attn_mask_per_layer = torch.randint(1, 11, (_N_H, TDST_BQ, MASK_K_BK, MODEL_N))

# Define the new tensor with size (N_H, timedim_leftover, MASK_K_BK, MODEL_N)
timedim_leftover = 3  # Example size
new_tensor = torch.full((_N_H, timedim_leftover, MASK_K_BK, MODEL_N), 9999999)

# Concatenate along the TDST_BQ dimension
# Dimension index for TDST_BQ is 1
result = torch.cat((ensemble_attn_mask_per_layer, new_tensor), dim=1)

print(ensemble_attn_mask_per_layer.shape)
print(ensemble_attn_mask_per_layer)
print(new_tensor.shape)
print(result)
# Check the new shape of the resulting tensor
print("New tensor shape:", result.shape)


torch.Size([3, 4, 2, 5])
tensor([[[[ 3,  3,  7,  2, 10],
          [ 6,  1,  9,  8,  3]],

         [[ 1,  5,  9,  1,  9],
          [ 7,  4,  2,  9,  4]],

         [[ 7,  9,  5,  9,  2],
          [ 3,  6,  3,  9, 10]],

         [[ 5,  6, 10,  2,  9],
          [ 1, 10,  2,  2,  7]]],


        [[[ 6,  2,  9,  2,  2],
          [ 8,  7,  9, 10, 10]],

         [[10,  6,  3,  2,  2],
          [ 4,  3,  5,  8,  6]],

         [[ 4,  1,  6,  1,  5],
          [ 9,  3, 10,  4,  1]],

         [[ 8,  3,  6,  9,  2],
          [ 8,  7,  8,  9,  6]]],


        [[[ 4,  3,  5,  8, 10],
          [ 2,  3,  8,  1,  2]],

         [[ 6,  3, 10,  9,  2],
          [ 3, 10,  8,  9,  8]],

         [[ 8,  1,  4,  8,  9],
          [ 9,  9,  6,  8,  2]],

         [[ 2,  2,  1,  2,  2],
          [ 1,  9,  4, 10,  7]]]])
torch.Size([3, 3, 2, 5])
tensor([[[[      3,       3,       7,       2,      10],
          [      6,       1,       9,       8,       3]],

         [[      1,       5,       9,

In [9]:
ensemble_attn_mask_per_layer

tensor([[ 3,  3,  4,  8,  8,  9,  4,  9,  9,  4,  7,  2],
        [ 6, 10,  8,  4,  1,  9,  7, 10, 10,  4,  2,  4],
        [ 9,  8,  3,  7,  9,  5,  2, 10,  2,  4,  1,  8],
        [ 1,  6,  8,  4, 10,  9,  4,  1,  4,  2, 10,  9],
        [ 3,  6,  5,  7,  7,  2,  8,  2,  8,  7, 10,  8],
        [ 9,  4,  2,  8,  1,  7,  4,  7,  8,  5,  2, 10],
        [10,  7,  2,  1,  2,  9,  4,  3, 10,  4,  1,  9],
        [ 1,  6,  6,  6,  6,  4,  2,  7, 10,  9,  2,  9],
        [ 3,  5,  2,  9,  6,  5,  3,  2,  5,  6,  6,  9],
        [10,  4,  4,  2,  2,  8,  9,  8,  9,  3,  4,  9],
        [ 6,  8,  8,  4,  9,  8,  9,  1,  9, 10,  6,  3],
        [ 8,  8,  6,  8,  4,  9,  4,  1, 10,  4,  5,  3],
        [ 4, 10,  8,  8,  9,  8,  4,  2,  8,  3,  7, 10],
        [10,  3,  5,  4,  6,  6,  6, 10,  5, 10,  6,  9],
        [ 9,  8,  7,  9,  7,  3,  6,  1, 10,  1,  1,  8]])

In [10]:
result

tensor([[9999999,       2,       3,       4, 9999999, 9999999,       7,       8,
               9, 9999999, 9999999, 9999999],
        [      1,       2, 9999999,       4, 9999999,       6,       7,       8,
               9,      10, 9999999, 9999999],
        [      1,       2,       3,       4,       5, 9999999,       7,       8,
               9,      10, 9999999, 9999999],
        [      1,       2, 9999999,       4, 9999999,       6, 9999999,       8,
               9,      10, 9999999, 9999999],
        [9999999,       2,       3, 9999999,       5,       6,       7,       8,
         9999999,      10, 9999999, 9999999],
        [      1,       2, 9999999,       4,       5, 9999999,       7,       8,
               9,      10, 9999999, 9999999],
        [      1,       2,       3,       4, 9999999, 9999999,       7, 9999999,
               9,      10, 9999999, 9999999],
        [      1,       2, 9999999,       4, 9999999,       6,       7, 9999999,
               9,      10, 999

In [11]:
result_cnt

tensor([[-9999999,        1,        2,        3, -9999999, -9999999,        1,
                2,        3, -9999999, -9999999, -9999999],
        [       1,        1, -9999999,        3, -9999999,        1,        1,
                1,        1,        3, -9999999, -9999999],
        [       1,        2,        1,        1,        1, -9999999,        1,
                2,        2,        1, -9999999, -9999999],
        [       2,        1, -9999999,        3, -9999999,        1, -9999999,
                1,        2,        2, -9999999, -9999999],
        [-9999999,        2,        1, -9999999,        1,        1,        3,
                3, -9999999,        1, -9999999, -9999999],
        [       1,        2, -9999999,        2,        1, -9999999,        2,
                2,        1,        1, -9999999, -9999999],
        [       2,        2,        1,        2, -9999999, -9999999,        1,
         -9999999,        2,        2, -9999999, -9999999],
        [       1,        2

In [12]:
ensemble_cnt_sorted, indices = torch.sort(result_cnt, dim=-1, descending=True)
ensemble_sorted = result.gather(-1, indices)
# mask = ensemble_cnt_sorted >= ensemble_method_final_inter_thresh

# ensemble_filtered = torch.where(mask, ensemble_sorted, torch.tensor(9999999, device=mask.device))
# ensemble_cnt_filtered = torch.where(mask, ensemble_cnt_sorted, torch.tensor(9999999, device=mask.device))


In [13]:
ensemble_sorted

tensor([[      4,       9,       3,       8,       2,       7, 9999999, 9999999,
         9999999, 9999999, 9999999, 9999999],
        [      4,      10,       1,       2,       6,       7,       8,       9,
         9999999, 9999999, 9999999, 9999999],
        [      2,       8,       9,       1,       3,       4,       5,       7,
              10, 9999999, 9999999, 9999999],
        [      4,       1,       9,      10,       2,       6,       8, 9999999,
         9999999, 9999999, 9999999, 9999999],
        [      7,       8,       2,       3,       5,       6,      10, 9999999,
         9999999, 9999999, 9999999, 9999999],
        [      2,       4,       7,       8,       1,       5,       9,      10,
         9999999, 9999999, 9999999, 9999999],
        [      1,       2,       4,       9,      10,       3,       7, 9999999,
         9999999, 9999999, 9999999, 9999999],
        [      6,       2,       9,       1,       4,       7,      10, 9999999,
         9999999, 9999999, 999

In [14]:
ensemble_cnt_sorted

tensor([[       3,        3,        2,        2,        1,        1, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       3,        3,        1,        1,        1,        1,        1,
                1, -9999999, -9999999, -9999999, -9999999],
        [       2,        2,        2,        1,        1,        1,        1,
                1,        1, -9999999, -9999999, -9999999],
        [       3,        2,        2,        2,        1,        1,        1,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       3,        3,        2,        1,        1,        1,        1,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       2,        2,        2,        2,        1,        1,        1,
                1, -9999999, -9999999, -9999999, -9999999],
        [       2,        2,        2,        2,        2,        1,        1,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       4,        2

In [3]:
import torch
import math
_N_H = 2
TDST_BQ = 5
MASK_K_BK = 4
MODEL_N = 3

ensemble_attn_mask_per_layer = torch.randn(_N_H, TDST_BQ, MASK_K_BK, MODEL_N)

ensemble_attn_mask_per_layer = ensemble_attn_mask_per_layer.view(_N_H * TDST_BQ, MASK_K_BK * MODEL_N)
# ensemble_attn_mask_per_layer : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
unique_x, indices, unique_cnt = torch.unique(ensemble_attn_mask_per_layer, return_inverse=True, sorted=False, return_counts=True)
# indices -= indices.min(dim=1, keepdims=True)[0]

# cnt_x = (unique_x[None, None, :] == ensemble_attn_mask_per_layer[:, :, None]).long().sum(1)
N, K = ensemble_attn_mask_per_layer.shape

cnt_xs = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    cnt_xs.append(
        (unique_x[None, None, :] == ensemble_attn_mask_per_layer[icn*chunk_n:icn*chunk_n+chunk_n, :, None]).long().sum(1)
    )
cnt_x = torch.cat(cnt_xs, dim=0)


result = torch.full_like(ensemble_attn_mask_per_layer, 9999999)
result = result.scatter_(1, indices.clamp(0, K-1), ensemble_attn_mask_per_layer)


# t = result[:, :, None] == unique_x[None, None, :]
# t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
# t = t.sum(-1)
N, K = result.shape
ts = []
chunk_n = 64
for icn in range(int(math.ceil(N / chunk_n))):
    t = result[icn*chunk_n:icn*chunk_n+chunk_n, :, None] == unique_x[None, None, :]
    t = t * torch.arange(len(unique_x), device=t.device)[None, None, :]
    t = t.sum(-1)
    ts.append(t)
t = torch.cat(ts, dim=0)

# torch.Size([4096, 1280]) torch.Size([64, 1280]) torch.Size([4096, 2094])
# print(result.shape, t.shape, cnt_x.shape)
result_cnt = torch.where(result < 9999999, cnt_x.gather(-1, t), -9999999)

'''
ensemble_attn_mask_per_layer
tensor([[1, 1, 3, 3, 3, 5],\
        [3, 3, 4, 4, 4, 4]])
ensemble_cnt_sorted
tensor([[       6,        4,        3,        3,        2,        1,        1,
-9999999, -9999999, -9999999, -9999999, -9999999, -9999999, -9999999,
-9999999, -9999999, -9999999, -9999999, -9999999, -9999999],\
[       6,        4,        3,        3,        2,        2, -9999999,
-9999999, -9999999, -9999999, -9999999, -9999999, -9999999, -9999999,
-9999999, -9999999, -9999999, -9999999, -9999999, -9999999]])
ensemble_sorted
tensor([[   10,     6,     3,     8,     1,     5,    11, 32000, 32000, 32000,
32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000],\
[    5,     4,     7,     9,     6,     3, 32000, 32000, 32000, 32000,
32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000, 32000]]))
'''
# ensemble_sorted : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
# ensemble_cnt_sorted : [N*H * TDST//BLOCK_SIZE_Q, mask_k//BLOCK_SIZE_K * ensemble_model_n]
ensemble_cnt_sorted, indices = torch.sort(result_cnt, dim=-1, descending=True)
ensemble_sorted = result.gather(-1, indices)

In [4]:
ensemble_cnt_sorted.shape, ensemble_sorted.shape

(torch.Size([10, 12]), torch.Size([10, 12]))

In [5]:
ensemble_cnt_sorted

tensor([[       1,        1, -9999999, -9999999, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1, -9999999, -9999999, -9999999, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1,        1, -9999999, -9999999, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1, -9999999, -9999999, -9999999, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1,        1,        1, -9999999, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1,        1, -9999999, -9999999, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1,        1,        1,        1, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1,        1

In [6]:
t = ensemble_cnt_sorted

In [7]:
t

tensor([[       1,        1, -9999999, -9999999, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1, -9999999, -9999999, -9999999, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1,        1, -9999999, -9999999, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1, -9999999, -9999999, -9999999, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1,        1,        1, -9999999, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1,        1, -9999999, -9999999, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1,        1,        1,        1, -9999999, -9999999, -9999999,
         -9999999, -9999999, -9999999, -9999999, -9999999],
        [       1,        1

In [8]:
id(ensemble_cnt_sorted), id(t)

(129188242076432, 129188242076432)